In [49]:
import pandas as pd
import re
import hashlib
from langdetect import detect, DetectorFactory
import numpy as np
from tqdm import tqdm
import unicodedata

# Configuración para detección de idioma consistente
DetectorFactory.seed = 0

# Configuración de pandas para mostrar más columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

In [50]:
# Cargar el dataset
df = pd.read_csv('limpiar_enalces.csv')

# Mostrar información inicial del dataset
print("\033[1mInformación inicial del dataset:\033[0m")
print(f"Registros totales: {len(df)}")
print(f"Columnas originales: {df.columns.tolist()}")
print("\nPrimeras filas del dataset:")
display(df.head(3))

Información inicial del dataset:
Registros totales: 4052
Columnas originales: ['Tweet_ID', 'Username', 'Text', 'Created_At', 'Location_Mentioned', 'tweet_length']

Primeras filas del dataset:


,Tweet_ID,Username,Text,Created_At,Location_Mentioned,tweet_length
0,1828598353636086250,SAJO,Se fue la luz y se dañó el panel electrónico del elevador y el AC. Luma nos está destruyendo y cobrando por hacerlo. Somos muchos los afectados. El desquite va. 👊🏻👊🏻👊🏻,2024-07-22,Quito,167
1,1788212151393423687,🌸♡🌸Elizαβετh🌸♡🌸,"No quiero arrecharme, pero estos HDP de Corpoelec quitan la luz hoy q hay agua y se puede lavar!!! 😤\n\nCdo hay electricidad, no hay agua \nCdo hay agua, no hay electricidad.\nPónganse d acuerdo n...",2024-12-01,Azuay,257
2,1801749927090847994,Jay Fonseca,"🌌 A dos días del apagón masivo que dejó a medio archipiélago a oscuras, ciudadanos reclamaron esta tarde la salida de LUMA Energy y de Genera PR, los dos operadores privados del sistema eléctrico....",2024-10-15,Guayaquil,299


In [51]:
# Función para hashear el Tweet_ID
def hashear_id(tweet_id):
    if pd.isna(tweet_id):
        return None
    return hashlib.sha256(str(tweet_id).encode()).hexdigest()

# Aplicar hashing al Tweet_ID
print("\033[1mHasheando Tweet_ID...\033[0m")
df['Tweet_ID'] = df['Tweet_ID'].apply(hashear_id)

# Eliminar la columna Username
print("\033[1mEliminando columna Username...\033[0m")
df.drop(columns=['Username'], inplace=True)

# Verificar cambios
print("\n\033[1mDataset después de hashear y eliminar:\033[0m")
display(df.head(3))

Hasheando Tweet_ID...
Eliminando columna Username...

Dataset después de hashear y eliminar:


,Tweet_ID,Text,Created_At,Location_Mentioned,tweet_length
0,c3c923bab9a5006041fee2df8da9a2173b2de335f62342eebe05f0208c97e6c1,Se fue la luz y se dañó el panel electrónico del elevador y el AC. Luma nos está destruyendo y cobrando por hacerlo. Somos muchos los afectados. El desquite va. 👊🏻👊🏻👊🏻,2024-07-22,Quito,167
1,23cf0973031f6388a6bacb5306160f57e5f23cbdbb597cdb6726ca6e459aa3e5,"No quiero arrecharme, pero estos HDP de Corpoelec quitan la luz hoy q hay agua y se puede lavar!!! 😤\n\nCdo hay electricidad, no hay agua \nCdo hay agua, no hay electricidad.\nPónganse d acuerdo n...",2024-12-01,Azuay,257
2,33da3cb8621b2ce82a4eb6441dfabf4a059ddde7348fadebdb78a4b11370728a,"🌌 A dos días del apagón masivo que dejó a medio archipiélago a oscuras, ciudadanos reclamaron esta tarde la salida de LUMA Energy y de Genera PR, los dos operadores privados del sistema eléctrico....",2024-10-15,Guayaquil,299


In [52]:
def limpiar_texto_mejorado(texto):
    if pd.isna(texto) or texto == '':
        return ""
    
    # Convertir a string por si acaso
    texto = str(texto)
    
    # 1. Eliminar saltos de línea y unir todo en una línea
    texto = re.sub(r'\n|\r|\t', ' ', texto)
    
    # 2. Eliminar enlaces (http, https, t.co, etc.)
    texto = re.sub(r'http\S+|www\S+|https?://\S+|t\.co/\S+', '', texto, flags=re.IGNORECASE)
    
    # 3. Eliminar menciones (@usuario)
    texto = re.sub(r'@\w+', '', texto)
    
    # 4. Eliminar completamente el hashtag y todo lo que le sigue
    texto = re.sub(r'#\S+', '', texto)  # 
    
    # 5. Eliminar emojis y símbolos especiales (patrón ampliado)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticonos
        u"\U0001F300-\U0001F5FF"  # símbolos & pictogramas
        u"\U0001F680-\U0001F6FF"  # transporte & símbolos
        u"\U0001F700-\U0001F77F"  # alquimia
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251" 
        u"\U0001F004-\U0001F0CF"
        u"\U0001F170-\U0001F251"
        "•→←↑↓«»⏰"  # Símbolos adicionales específicos
        "]+", flags=re.UNICODE)
    texto = emoji_pattern.sub(r'', texto)
    
    # 6. Eliminar caracteres especiales pero conservar signos de puntuación básicos
    caracteres_especiales = r'[|*{}\[\]()/&%"¬=°$\'¿¡~+]'
    texto = re.sub(caracteres_especiales, ' ', texto)
    
    # 7. Eliminar números
    texto = re.sub(r'\d+', '', texto)
    
    # 8. Convertir a minúsculas
    texto = texto.lower()
    
    # 9. Eliminar puntos suspensivos, comillas y patrones especiales
    texto = re.sub(r'\.{2,}', ' ', texto)  # Puntos suspensivos
    texto = re.sub(r'…', ' ', texto)  # Puntos suspensivos unicode
    texto = re.sub(r'´|`', '', texto)  # Acentos sueltos
    
    # 10. Eliminar comillas de todo tipo (incluyendo las angulares)
    comillas = r'[\'\"\‘\’\“\”\´\`«»]'
    texto = re.sub(comillas, '', texto)
    
    # 11. Normalizar espacios múltiples y bordes
    texto = re.sub(r'\s{2,}', ' ', texto)  # Espacios múltiples
    texto = texto.strip()
    
    # 13. Eliminar palabras sueltas muy cortas (1-2 letras) que no aportan significado
    palabras_relevantes = ['no', 'si', 'se', 'me', 'te', 'le', 'lo', 'la', 'los', 'las', 
                          'un', 'una', 'uno', 'unos', 'unas', 'al', 'del', 'él', 'ella']
    texto = ' '.join([word for word in texto.split() 
                     if len(word) > 2 or word in palabras_relevantes])
    
    return texto

# Función para detectar texto válido (no vacío después de limpieza)
def es_texto_valido(texto):
    texto_limpio = limpiar_texto_mejorado(texto)
    return len(texto_limpio.strip()) > 3  # Consideramos válido si tiene más de 3 caracteres

# Aplicar limpieza mejorada con barra de progreso
print("\033[1mProcesando limpieza de texto...\033[0m")
tqdm.pandas()
df['Text_Clean'] = df['Text'].progress_apply(limpiar_texto_mejorado)

# Filtrar solo textos válidos
df_filtrado = df[df['Text'].apply(es_texto_valido)].copy()
print(f"\033[1mRegistros después de limpieza:\033[0m {len(df_filtrado)}")
print(f"\033[1mRegistros eliminados:\033[0m {len(df) - len(df_filtrado)}")

Procesando limpieza de texto...


100%|██████████| 4052/4052 [00:00<00:00, 19478.76it/s]


Registros después de limpieza: 4052
Registros eliminados: 0


In [53]:
# Función mejorada para detección de idioma con manejo de errores
def es_espanol_mejorado(texto):
    try:
        # Si el texto es muy corto, no podemos detectar idioma confiablemente
        if len(texto) < 10:
            return True  # Asumimos que es español para no perder datos
        
        # Detectar idioma
        return detect(texto) == 'es'
    except:
        return True  # En caso de error, lo mantenemos para no perder datos

# Filtrar por idioma español (con enfoque conservador para no perder datos)
print("\033[1mFiltrando por idioma español...\033[0m")
df_final = df_filtrado[df_filtrado['Text_Clean'].apply(es_espanol_mejorado)].copy()
print(f"\033[1mRegistros finales:\033[0m {len(df_final)}")
print(f"\033[1mRegistros eliminados por idioma:\033[0m {len(df_filtrado) - len(df_final)}")

Filtrando por idioma español...
Registros finales: 3815
Registros eliminados por idioma: 237


In [56]:
import pandas as pd
import re
from tqdm import tqdm

# Configuración para evitar warnings
pd.set_option('mode.chained_assignment', None)

# Función de limpieza mejorada
def limpiar_texto_final(texto):
    if pd.isna(texto) or texto == '':
        return ""
    
    texto = str(texto)
    
    # 1. Corregir letras repetidas (sin grupos problemáticos)
    texto = re.sub(r'([a-zA-ZáéíóúüñÁÉÍÓÚÜÑ])\1{2,}', lambda m: m.group(1), texto)
    
    # 2. Eliminar caracteres especiales excepto tildes y ñ
    texto = re.sub(r'[^\w\sáéíóúüñÁÉÍÓÚÜÑ]', '', texto)
    
    # 3. Normalizar espacios
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto

# Crear copia explícita para evitar SettingWithCopyWarning
df_final_clean = df_final.copy()

# Aplicar limpieza
tqdm.pandas()
df_final_clean.loc[:, 'Text_Final'] = df_final_clean['Text_Clean'].progress_apply(limpiar_texto_final)

# Lista de palabras en portugués a filtrar
palabras_portugues = [
    'obrigado', 'obrigada', 'por favor', 'você', 'sim', 'não', 
    'com licença', 'bom dia', 'boa tarde', 'boa noite', 'louça'
]

# Filtrar registros en portugués (sin regex para evitar warnings)
df_final_espanol = df_final_clean[
    ~df_final_clean['Text_Final'].str.lower().str.contains('|'.join(palabras_portugues), na=False)
]

# Patrones de verificación (sin grupos de captura)
noise_patterns_final = {
    'URLs residuales': r'https?:|www\.',
    'HTML tags': r'&lt;|&gt;|<[a-z]+>',
    'Puntuación múltiple': r'![!]+|\?[\?]+|\.{2,}',
    'Caracteres especiales no permitidos': r'[^a-z0-9\sáéíóúüñ]',
    'Números residuales': r'\b\d\d+\b',
    'Letras repetidas': r'([a-záéíóúüñ])\1\1'
}

print("\n\033[1mVerificación Final de Ruido Residual:\033[0m")

# Función segura para contar patrones (sin warnings)
def contar_patrones_seguro(serie, pattern):
    try:
        # Convertir a minúsculas y usar str.count() en lugar de contains()
        return serie.str.lower().str.count(pattern).sum()
    except:
        return 0

noise_counts = {}
for name, pattern in noise_patterns_final.items():
    count = contar_patrones_seguro(df_final_espanol['Text_Final'], pattern)
    noise_counts[name] = count
    print(f"{name}: {count} tweets ({count/len(df_final_espanol)*100:.2f}%)")

# Mostrar solo resultados sin ejemplos específicos
print("\n\033[1;32m¡Proceso completado exitosamente!\033[0m")
print(f"Total tweets procesados: {len(df_final_espanol)}")

100%|██████████| 3815/3815 [00:00<00:00, 47132.15it/s]


Verificación Final de Ruido Residual:
URLs residuales: 0 tweets (0.00%)
HTML tags: 0 tweets (0.00%)
Puntuación múltiple: 0 tweets (0.00%)
Caracteres especiales no permitidos: 7 tweets (0.19%)
Números residuales: 0 tweets (0.00%)
Letras repetidas: 0 tweets (0.00%)

¡Proceso completado exitosamente!
Total tweets procesados: 3676


In [59]:
# Seleccionar solo las columnas requeridas
columnas_finales = ['Tweet_ID', 'Created_At', 'Location_Mentioned', 'tweet_length', 'Text', 'Text_Final']
df_exportar = df_final_espanol[columnas_finales].copy()

# Verificar las columnas seleccionadas
print("\033[1mColumnas en el dataset final:\033[0m")
print(df_exportar.columns.tolist())

# Verificar ejemplo de datos
print("\n\033[1mMuestra del dataset final:\033[0m")
display(df_exportar.head(2))

# Guardar a CSV (versión compatible con todos los caracteres)
nombre_archivo = 'tweets_procesados_final.csv'
df_exportar.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')

print(f"\n\033[1;32mDataset exportado exitosamente como:\033[0m {nombre_archivo}")
print(f"Total de registros exportados: {len(df_exportar)}")

Columnas en el dataset final:
['Tweet_ID', 'Created_At', 'Location_Mentioned', 'tweet_length', 'Text', 'Text_Final']

Muestra del dataset final:


,Tweet_ID,Created_At,Location_Mentioned,tweet_length,Text,Text_Final
0,c3c923bab9a5006041fee2df8da9a2173b2de335f62342eebe05f0208c97e6c1,2024-07-22,Quito,167,Se fue la luz y se dañó el panel electrónico del elevador y el AC. Luma nos está destruyendo y cobrando por hacerlo. Somos muchos los afectados. El desquite va. 👊🏻👊🏻👊🏻,se fue la luz se dañó panel electrónico del elevador ac luma nos está destruyendo cobrando por hacerlo somos muchos los afectados desquite va
1,23cf0973031f6388a6bacb5306160f57e5f23cbdbb597cdb6726ca6e459aa3e5,2024-12-01,Azuay,257,"No quiero arrecharme, pero estos HDP de Corpoelec quitan la luz hoy q hay agua y se puede lavar!!! 😤\n\nCdo hay electricidad, no hay agua \nCdo hay agua, no hay electricidad.\nPónganse d acuerdo n...",no quiero arrecharme pero estos hdp corpoelec quitan la luz hoy hay agua se puede lavar cdo hay electricidad no hay agua cdo hay agua no hay electricidad pónganse acuerdo nojodas tenía desahogarme...



Dataset exportado exitosamente como: tweets_procesados_final.csv
Total de registros exportados: 3676
